In [1]:
#working better
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests


dict_list = []

driver_path = 'C:/Users/liman/Desktop/chromedriver_win32/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(driver_path, options=options)

for page in range(1, 314):
    url = f"https://rieltor.ua/kiev/flats-rent/?page={page}"
    driver.get(url)
    driver.implicitly_wait(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for i in soup.find_all('div', class_="catalog-card"):
        flat_dict = {}  # create a new dictionary for each iteration
        
        price = i.get('data-label')
        price_num = int(''.join(filter(str.isdigit, price)))
        
        price_currency = price[-1]
        price_currency
        
        lat = i.get('data-latitude')
        long = i.get('data-longitude')
        address = i.find("div", {"class": "catalog-card-address"}).text
        room_type = i.find('div', class_='catalog-card-details-row').span.text.strip()

        #district of Kyiv   
        region_links = i.find_all('a', {'data-analytics-event': 'card-click-region'})
        district = region_links[1].text.strip()

        #calculate area of the flat
        area = i.find_all('div', class_='catalog-card-details-row')[-2].span.text
        area_str = area.replace(" м²", "")
        area_list = area_str.split(" / ")
        area_list = [float(x) for x in area_list]
        area_total = sum(area_list)

        #starting dict
        flat_dict["rooms"] = room_type
        flat_dict["price"] = price_num
        flat_dict["initial_currency"] = price_currency
        flat_dict["flat_area"] = area_total
        flat_dict["district"] = district
        flat_dict["address"] = address
        flat_dict["latitude"] = lat
        flat_dict["longitude"] = long
        

        #subway presence    
        subway_element = i.find("a", {"class": "catalog-card-chip -subway -subway-blue"})
        if subway_element is not None:
            subway_station = subway_element.span.text
        else:
            subway_station = None

        flat_dict["subway"] = subway_station

        dict_list.append(flat_dict)

# print(dict_list)

C:\Users\liman\AppData\Local\Temp\ipykernel_12368\1073210370.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path, options=options)


IndexError: list index out of range

In [2]:
df = pd.DataFrame.from_dict(dict_list)
df

,rooms,price,initial_currency,flat_area,district,address,latitude,longitude,subway
0,2 кімнати,25000,н,131.0,Голосіївський р-н,"Рудницького Степана вул. (Вільямса академіка),...",50.38436508,30.4560318,Іподром
1,2 кімнати,15500,н,114.0,Солом'янський р-н,"Лобановського просп. (Червонозоряний), 4в",50.42233276,30.4624939,None
2,1 кімната,7000,н,55.0,Оболонський р-н,"Оболонський просп., 43",50.52085495,30.49764442,Героїв Дніпра
3,2 кімнати,20000,н,117.0,Печерський р-н,"Лесі Українки бульв., 5",50.43521118,30.53017426,None
4,3 кімнати,20000,н,118.0,Печерський р-н,"Лесі Українки бульв., 5",50.43521118,30.53017426,None
...,...,...,...,...,...,...,...,...,...
5947,1 кімната,15000,н,98.0,Голосіївський р-н,"Феодосійський пров., 14",50.40266418,30.53172684,Деміївська
5948,1 кімната,8000,н,43.0,Святошинський р-н,"Туполєва академіка вул., 7Б",50.48032379,30.39466858,None
5949,2 кімнати,15000,н,107.0,Святошинський р-н,"Клавдієвська вул., 40 г",50.47770691,30.33908081,None
5950,4 кімнати,950,$,194.5,Печерський р-н,"Інститутська вул., 16",50.44618607,30.5323143,None


In [3]:
df.shape

(5952, 9)

In [4]:
#drop duplicates by address 
df_unique = df.drop_duplicates(subset='address', keep='first').reset_index(drop=True)
df_unique

,rooms,price,initial_currency,flat_area,district,address,latitude,longitude,subway
0,2 кімнати,25000,н,131.0,Голосіївський р-н,"Рудницького Степана вул. (Вільямса академіка),...",50.38436508,30.4560318,Іподром
1,2 кімнати,15500,н,114.0,Солом'янський р-н,"Лобановського просп. (Червонозоряний), 4в",50.42233276,30.4624939,None
2,1 кімната,7000,н,55.0,Оболонський р-н,"Оболонський просп., 43",50.52085495,30.49764442,Героїв Дніпра
3,2 кімнати,20000,н,117.0,Печерський р-н,"Лесі Українки бульв., 5",50.43521118,30.53017426,None
4,3 кімнати,12000,н,140.0,Дарницький р-н,"Драгоманова вул., 25",50.40409088,30.63778687,None
...,...,...,...,...,...,...,...,...,...
3421,3 кімнати,18000,н,212.0,Дніпровський р-н,Всеволода Нестайка вул. (Мильчакова Олександра...,50.45552826,30.59948158,None
3422,2 кімнати,10000,н,110.0,Дарницький р-н,"Вишняківська вул., 13б",50.39374924,30.64518547,None
3423,1 кімната,8000,н,43.0,Святошинський р-н,"Туполєва академіка вул., 7Б",50.48032379,30.39466858,None
3424,2 кімнати,15000,н,107.0,Святошинський р-н,"Клавдієвська вул., 40 г",50.47770691,30.33908081,None


In [5]:
df_unique.shape

(3426, 9)

In [ ]:
# we should convert prices in dollars to prices in national ('н') currency

In [6]:
df_unique['price'] = df_unique.apply(lambda x: x['price'] * 36.94 if x['initial_currency'] == '$' else x['price'], axis=1)
df_unique


,rooms,price,initial_currency,flat_area,district,address,latitude,longitude,subway
0,2 кімнати,25000.0,н,131.0,Голосіївський р-н,"Рудницького Степана вул. (Вільямса академіка),...",50.38436508,30.4560318,Іподром
1,2 кімнати,15500.0,н,114.0,Солом'янський р-н,"Лобановського просп. (Червонозоряний), 4в",50.42233276,30.4624939,None
2,1 кімната,7000.0,н,55.0,Оболонський р-н,"Оболонський просп., 43",50.52085495,30.49764442,Героїв Дніпра
3,2 кімнати,20000.0,н,117.0,Печерський р-н,"Лесі Українки бульв., 5",50.43521118,30.53017426,None
4,3 кімнати,12000.0,н,140.0,Дарницький р-н,"Драгоманова вул., 25",50.40409088,30.63778687,None
...,...,...,...,...,...,...,...,...,...
3421,3 кімнати,18000.0,н,212.0,Дніпровський р-н,Всеволода Нестайка вул. (Мильчакова Олександра...,50.45552826,30.59948158,None
3422,2 кімнати,10000.0,н,110.0,Дарницький р-н,"Вишняківська вул., 13б",50.39374924,30.64518547,None
3423,1 кімната,8000.0,н,43.0,Святошинський р-н,"Туполєва академіка вул., 7Б",50.48032379,30.39466858,None
3424,2 кімнати,15000.0,н,107.0,Святошинський р-н,"Клавдієвська вул., 40 г",50.47770691,30.33908081,None


In [7]:
#and now we can drop the column 'initial_currency'
df_unique = df_unique.drop('initial_currency', axis=1)
df_unique

,rooms,price,flat_area,district,address,latitude,longitude,subway
0,2 кімнати,25000.0,131.0,Голосіївський р-н,"Рудницького Степана вул. (Вільямса академіка),...",50.38436508,30.4560318,Іподром
1,2 кімнати,15500.0,114.0,Солом'янський р-н,"Лобановського просп. (Червонозоряний), 4в",50.42233276,30.4624939,None
2,1 кімната,7000.0,55.0,Оболонський р-н,"Оболонський просп., 43",50.52085495,30.49764442,Героїв Дніпра
3,2 кімнати,20000.0,117.0,Печерський р-н,"Лесі Українки бульв., 5",50.43521118,30.53017426,None
4,3 кімнати,12000.0,140.0,Дарницький р-н,"Драгоманова вул., 25",50.40409088,30.63778687,None
...,...,...,...,...,...,...,...,...
3421,3 кімнати,18000.0,212.0,Дніпровський р-н,Всеволода Нестайка вул. (Мильчакова Олександра...,50.45552826,30.59948158,None
3422,2 кімнати,10000.0,110.0,Дарницький р-н,"Вишняківська вул., 13б",50.39374924,30.64518547,None
3423,1 кімната,8000.0,43.0,Святошинський р-н,"Туполєва академіка вул., 7Б",50.48032379,30.39466858,None
3424,2 кімнати,15000.0,107.0,Святошинський р-н,"Клавдієвська вул., 40 г",50.47770691,30.33908081,None


In [13]:
df_unique.to_csv('orenda_Kyiv_lun_rieltor.csv', index=False)

In [14]:
df_unique.to_excel('orenda_Kyiv_lun_rieltor.xlsx', index=False)